In [1]:
### Code for reproducing COVID-19 experiments found in https://arxiv.org/abs/2208.08401. The prediction method and associated
### code used here is adapted from work by Ryan Tibshirani https://delphi.cmu.edu/blog/2020/09/21/can-symptoms-surveys-improve-covid-19-forecasts/

library(covidcast)
library(dplyr)
library(tidyr)
library(ggplot2)
source("AgACI.R")
source("DtACI.R")
source("MVP.R")

We encourage COVIDcast API users to register on our mailing list:
https://lists.andrew.cmu.edu/mailman/listinfo/delphi-covidcast-api
We'll send announcements about new data sources, package updates,
server maintenance, and new features.


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




In [2]:
################# Construct the Dataset #########################

# Function to append shift values (lags or leads) to data frame
append_shifts = function(df, shifts) {
  # Make sure that we have a complete record of dates for each geo_value (fill
  # with NAs as necessary)
  df_all = df %>% group_by(geo_value) %>%
    summarize(time_value = seq.Date(as.Date(min(time_value)),
                                    as.Date(max(time_value)),
                                    by = "day")) %>% ungroup()
  df = full_join(df, df_all, by = c("geo_value", "time_value"))
  
  # Group by geo value, sort rows by increasing time
  df = df %>% group_by(geo_value) %>% arrange(time_value)
  
  # Load over shifts, and add lag value or lead value
  for (shift in shifts) {
    fun = ifelse(shift < 0, dplyr::lag, dplyr::lead)
    varname = sprintf("value%+d", shift)
    df = mutate(df, !!varname := fun(value, n = abs(shift)))
  }
  # Ungroup and return
  return(ungroup(df))
}

# Some useful functions for transformations
Log = function(x, a = 0.01) log(x + a)
Exp = function(y, a = 0.01) exp(y) - a
Logit = function(x, a = 0.01) log((x + a) / (1 - x + a))
Sigmd = function(y, a = 0.01) (exp(y) * (1 + a) - a) / (1 + exp(y))
Id = function(x) x

#### Parameters #####

# Transforms to consider, in what follows
trans = Id
inv_trans = Id

# Rescale factors for our signals: bring them all down to proportions (between
# 0 and 1)
rescale_g = 1e-2 # Originally a percentage
rescale_f = 1e-2 # Originally a percentage
rescale_c = 1e-5 # Originally a count per 100,000 people

n = 14 # Number of trailing days to use for training set
lp_solver = "glpk" # LP solver to use in quantile_lasso()
verbose = TRUE # Print intermediate progress to console?

#### Data #####
geo_values = covidcast_signal("jhu-csse", "confirmed_cumulative_num",
                              "2020-05-14", "2020-05-14")  %>% pull(geo_value)

# Fetch county-level Facebook % CLI-in-community signals, and JHU
# confirmed case incidence proportion
start_day = "2020-04-11"
end_day = "2022-05-01"
f = covidcast_signal("fb-survey", "smoothed_hh_cmnty_cli",
                     start_day, end_day) %>%
  filter(geo_value %in% geo_values) %>%
  select(geo_value, time_value, value)
c = covidcast_signal("jhu-csse", "confirmed_7dav_incidence_prop",
                     start_day, end_day) %>%
  filter(geo_value %in% geo_values) %>%
  select(geo_value, time_value, value)

# Find "complete" counties, present in all three data signals at all times
geo_values_complete = intersect(f$geo_value,c$geo_value)

# Filter to complete counties, transform the signals, append 1-2 week lags to
# all three, and also 1-2 week leads to case rates
lags = 1 * c(-7,-14)
leads = 1 * 7
fm = f %>% filter(geo_value %in% geo_values_complete) %>%
  mutate(value = trans(value * rescale_f)) %>%
  append_shifts(shifts = lags)
cm = c %>% filter(geo_value %in% geo_values_complete) %>%
  mutate(value = trans(value * rescale_c)) %>%
  append_shifts(shifts = c(lags, leads))

# Rename columns
colnames(fm) = sub("^value", "fb", colnames(fm))
colnames(cm) = sub("^value", "case", colnames(cm))

# Make one big matrix by joining these two data frames
z = full_join(fm,cm, by = c("geo_value", "time_value"))

##### Analysis #####
res_list = vector("list", length = length(leads))

# Loop over lead, forecast dates, build models and record errors (warning: this
# computation takes a while)
for (i in 1:length(leads)) {
  lead = leads[i]; if (verbose) cat("***", lead, "***\n")
  
  # Create a data frame to store our forecast results. Code below populates its
  # rows in a way that breaks from typical dplyr operations, done for efficiency
  res_list[[i]] = z %>%
    filter(between(time_value, as.Date(start_day) - min(lags) + lead,
                   as.Date(end_day) - lead)) %>%
    select(geo_value, time_value) %>%
    mutate(err0 = as.double(NA),err1 = as.double(NA), lead = lead)
  valid_dates = unique(res_list[[i]]$time_value)
  
  for (k in 1:length(valid_dates)) {
    date = valid_dates[k]; if (verbose) cat(format(date), "... ")
    
    # Filter down to training set and test set
    z_tr = z %>% filter(between(time_value, date - lead - n, date - lead))
    z_te = z %>% filter(time_value == date)
    inds = which(res_list[[i]]$time_value == date)
    
    # Create training and test responses
    y_tr = z_tr %>% pull(paste0("case+", lead))
    y_te = z_te %>% pull(paste0("case+", lead))
    
    # Strawman model
    if (verbose) cat("0")
    y_hat = z_te %>% pull(case)
    res_list[[i]][inds,]$err0 = abs(inv_trans(y_hat) - inv_trans(y_te))
    
    # Cases and Facebook model
    x_tr_case = z_tr %>% select(starts_with("case") & !contains("+"))
    x_te_case = z_te %>% select(starts_with("case") & !contains("+"))
    x_tr_fb = z_tr %>% select(starts_with("fb"))
    x_te_fb = z_te %>% select(starts_with("fb"))
    x_tr = cbind(x_tr_case, x_tr_fb)
    x_te = cbind(x_te_case, x_te_fb)
    ok = complete.cases(x_tr, y_tr)
    if (sum(ok) > 0) {
      obj = lm(y_tr[ok] ~ as.matrix(x_tr[ok,]))
      y_hat = as.matrix(x_te)%*%(obj$coefficients[-1]) + obj$coefficients[1]
      res_list[[i]][inds,]$err1 = as.numeric(abs(inv_trans(y_hat) - inv_trans(y_te)))
    }
  }
}

# Bind results over different leads into one big data frame, and save
res = do.call(rbind, res_list)
#save(list = ls(), file = "demo.rda")

Fetched day 2020-05-14 to 2020-05-14: num_entries = 3282

Fetched day 2020-04-11 to 2022-05-01: num_entries = 427922

Warning message:
“Data not fetched for the following days: 2020-04-11”
Fetched day 2020-04-11 to 2021-02-09: num_entries = 998553

Fetched day 2021-02-10 to 2021-12-11: num_entries = 982464

Fetched day 2021-12-12 to 2022-05-01: num_entries = 453879

Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungrouped data frame and adjust accordingly.”
`summarise()` has grouped output by 'geo_value'. You can override using the
`.groups` argument.
Warning message:
“Returning more (or less) than 1 row per `summarise()` group was deprecated in
dplyr 1.1.0.
ℹ Please use `reframe()` instead.
ℹ When switching from `summarise()` to `reframe()`, remember that `reframe()`
  always returns an ungroup

*** 7 ***
2020-05-02 ... 02020-05-03 ... 02020-05-04 ... 02020-05-05 ... 02020-05-06 ... 02020-05-07 ... 02020-05-08 ... 02020-05-09 ... 02020-05-10 ... 02020-05-11 ... 02020-05-12 ... 02020-05-13 ... 02020-05-14 ... 02020-05-15 ... 02020-05-16 ... 02020-05-17 ... 02020-05-18 ... 02020-05-19 ... 02020-05-20 ... 02020-05-21 ... 02020-05-22 ... 02020-05-23 ... 02020-05-24 ... 02020-05-25 ... 02020-05-26 ... 02020-05-27 ... 02020-05-28 ... 02020-05-29 ... 02020-05-30 ... 02020-05-31 ... 02020-06-01 ... 02020-06-02 ... 02020-06-03 ... 02020-06-04 ... 02020-06-05 ... 02020-06-06 ... 02020-06-07 ... 02020-06-08 ... 02020-06-09 ... 02020-06-10 ... 02020-06-11 ... 02020-06-12 ... 02020-06-13 ... 02020-06-14 ... 02020-06-15 ... 02020-06-16 ... 02020-06-17 ... 02020-06-18 ... 02020-06-19 ... 02020-06-20 ... 02020-06-21 ... 02020-06-22 ... 02020-06-23 ... 02020-06-24 ... 02020-06-25 ... 02020-06-26 ... 02020-06-27 ... 02020-06-28 ... 02020-06-29 ... 02020-06-30 ... 02020-07-01 ... 02020-07-02 ...

In [3]:
########################## Compute Betas #################################

### Use a binary search to find the lowest quantile of recentScores that is above curScore
### Epsilon gives numerical error tolerance in the binary search
findBeta <- function(recentScores,curScore,epsilon=0.001){
  top <- 1
  bot <- 0
  mid <- (top+bot)/2
  while(top-bot > epsilon){
    if(quantile(recentScores,1-mid)>curScore){
      bot <- mid
      mid <- (top+bot)/2
    }else{
      top <- mid
      mid <- (top+bot)/2
    }
  }
  return(mid)
}

### Compute beta_ts for the given conformity scores. Lookback specifies the amount of time periods used to construct the model,
### epsilon specifies a numerical error tolerence for the binary search for beta_t,
### geosToUse specifies which counties we compute the betas for (NULL means look at all counties)
computeBetasByGeoByTime <- function(scores,lookback=1,epsilon=0.001,geosToUse = NULL){
  dates = unique(scores$time_value)
  geovals = unique(scores$geo_value)
  T <- length(dates)
  if(is.null(geosToUse)){
    geosToUse <- geovals
  }
  
  ### Initialize data storage variable
  betaSeqMat <- matrix(0,nrow = length(geosToUse),ncol=length(dates)-1)
  
  for(t in 2:length(dates)){
    prevScores <- scores$err1[scores$time_value==dates[t-1]]
    for(i in 1:length(geosToUse)){
      newScore <- scores$err1[scores$time_value==dates[t] & scores$geo_value==geosToUse[i]]
      if(length(newScore) > 0){
        betaSeqMat[i,t-1] <- findBeta(prevScores,newScore,epsilon)
      }else{
        betaSeqMat[i,t-1] <- NA
      }
    }
    if(t %% 10 == 0){
      print(sprintf("Done %g steps",t))
    }
  }
  
  return(betaSeqMat)
}

# Calculate the scaled errors relative to the strawman's error
res_final <- res %>%
  drop_na() %>%                                       # Restrict to common time
  mutate(err1 = err1 / err0) %>% 
  ungroup() %>%
  select(-err0)
res_final <- res_final %>% group_by(time_value) %>% arrange(geo_value) 

### Compute all beta_t values
alpha <- 0.1
gammaGrid <- c(0.001,0.002,0.004,0.008,0.0160,0.032,0.064,0.128)
geosToUse <- c("06075","36061","12086", "48113")  ## SF, NY, Miami-Dade, Dallas

allBetas <- computeBetasByGeoByTime(res_final,geosToUse = geosToUse)
rownames(allBetas) <- c("06075","36061","12086", "48113")


[1] "Done 10 steps"
[1] "Done 20 steps"
[1] "Done 30 steps"
[1] "Done 40 steps"
[1] "Done 50 steps"
[1] "Done 60 steps"
[1] "Done 70 steps"
[1] "Done 80 steps"
[1] "Done 90 steps"
[1] "Done 100 steps"
[1] "Done 110 steps"
[1] "Done 120 steps"
[1] "Done 130 steps"
[1] "Done 140 steps"
[1] "Done 150 steps"
[1] "Done 160 steps"
[1] "Done 170 steps"
[1] "Done 180 steps"
[1] "Done 190 steps"
[1] "Done 200 steps"
[1] "Done 210 steps"
[1] "Done 220 steps"
[1] "Done 230 steps"
[1] "Done 240 steps"
[1] "Done 250 steps"
[1] "Done 260 steps"
[1] "Done 270 steps"
[1] "Done 280 steps"
[1] "Done 290 steps"
[1] "Done 300 steps"
[1] "Done 310 steps"
[1] "Done 320 steps"
[1] "Done 330 steps"
[1] "Done 340 steps"
[1] "Done 350 steps"
[1] "Done 360 steps"
[1] "Done 370 steps"
[1] "Done 380 steps"
[1] "Done 390 steps"
[1] "Done 400 steps"
[1] "Done 410 steps"
[1] "Done 420 steps"
[1] "Done 430 steps"
[1] "Done 440 steps"
[1] "Done 450 steps"
[1] "Done 460 steps"
[1] "Done 470 steps"
[1] "Done 480 steps"
[

In [4]:
localMean <- function(seq,index,bandwidth,bottom=300){
  bottomIndex <- max(1,index-bottom+1)
  weights <- (bandwidth)^(bottomIndex:index)
  weights <- weights/sum(weights)
  return(sum(weights*seq[bottomIndex:index]))
}

makeCoveragePlot <- function(errMat,dates,alpha=0.1,smoothingWidth=500,cols = c("dtaci","agaci","mvp","split"),
                     xlabel = "Time",ylabel = "Coverage",colors=c(4,2,3,1),legendNames=cols,timeCutoff=0,lineSize=4,lineTyping=FALSE, alphaC = 0.8){
  T <- nrow(errMat)
  if(is.null(dates)){
      dates <- 1:T
  }
  errMatSmooth <- matrix(0,nrow=nrow(errMat),ncol=ncol(errMat))
  if(smoothingWidth>0){
    for(i in 1:length(cols)){
      errMatSmooth[,i] <- 1-sapply(1:T,function(x){localMean(errMat[,i],x,bandwidth=1,bottom=smoothingWidth)})
    }
  }
  colnames(errMatSmooth) <- cols
  finalMat <- as.data.frame(errMatSmooth)
  finalMat$Time <- dates
  finalMat <- pivot_longer(finalMat,cols=1:length(cols)) 
  colnames(finalMat) = c("Time","Method","Coverage")
  if(timeCutoff>0){
     finalMat <- finalMat[-(1:timeCutoff),]
  }
  if(lineTyping){
      p <- ggplot(finalMat,aes(x=Time, y=Coverage, group=Method,color=Method,linetype=Method))
  }else{
      p <- ggplot(finalMat,aes(x=Time, y=Coverage, group=Method,color=Method))
  }
  p <- p + geom_line(alpha=alphaC,size=lineSize) 
  p <- p + xlab(xlabel) + ylab(ylabel)  
  p <- p +  theme_bw() + theme(axis.text=element_text(size=40),axis.title=element_text(size=50,face="plain")) 
  p <- p + geom_hline(yintercept=1-alpha, linetype="dashed", color = "black",size = 4) 
  for(i in 1:ncol(errMat)){
     p <- p + geom_hline(yintercept=1-mean(errMat[,i]), linetype="dashed", color = colors[i],size = 4)   
  }
  return(p)
}


In [9]:
### Run DtACI, AgACI, and MVP, and Plot Results
library(latex2exp)
library(cowplot)
library(grid)
library(gridExtra)

alpha <- 0.1
gammaGrid <- c(0.001,0.002,0.004,0.008,0.0160,0.032,0.064,0.128)
Keps <- 2.12

myDates <- unique(res_final$time_value)[-1]
countiesToUse <- c("06075","36061","12086", "48113")  ## SF, NY, Miami-Dade, Dallas

allRes <- list()
allResAgACI <- list()
allResMVP <- list()
errMats <- list()
myPlots <- list()
count <- 1
for(geo in countiesToUse){
  allRes[[count]] <- conformalAdaptStable(allBetas[rownames(allBetas) ==geo],alpha,gammaGrid,sigma=1/1000,eta=2.72)
  allResAgACI[[count]] <- agaci(allBetas[rownames(allBetas) ==geo],alpha,gammaGrid)
  allResMVP[[count]] <- MVP(1-allBetas[rownames(allBetas) ==geo],alpha,Keps = Keps)
  errMats[[count]] <- cbind(allRes[[count]][[6]],allRes[[count]][[3]],allResMVP[[count]][[2]],allResAgACI[[count]][[2]])
  myPlots[[count]] <- makeCoveragePlot(errMats[[count]],myDates,alpha,cols=c("DtACI","Constant Baseline","MVP","AgACI"),
                                       colors=c(4,7,3,2),timeCutoff=200,smoothingWidth=200)
  myPlots[[count]] <- myPlots[[count]] + scale_colour_manual(name="Method",values = c("DtACI"=4,"AgACI"=2,"MVP"=3,"Constant Baseline"=7),
                                                             labels=c("DtACI","AgACI","MVP","Constant Baseline"),
                                                             aesthetics = c("color")) + ylim(0,1)
  count <- count + 1
}
for(i in 1:length(countiesToUse)){
  myPlots[[i]] <- myPlots[[i]] + theme(axis.text=element_text(size=40),axis.title=element_text(size=50,face="plain"))
}

legend <- get_legend(myPlots[[1]]+theme(legend.position = "top",legend.text=element_text(size=60),legend.title=element_blank(),
                                     legend.key.width = unit(3,"line")))
blankplot <- ggplot()+geom_blank(aes(1,1)) + cowplot::theme_nothing()
myPlots[[1]] <- myPlots[[1]] + ylim(0.5,1.05) +  annotate('text',x=as.Date("2020-11-08"), y=1.05, label="San Francisco, CA",size=18,fontface="bold") + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank()) + theme(legend.position = "none")
myPlots[[2]] <- myPlots[[2]] + ylim(0.5,1.05)  +  annotate('text',x=as.Date("2020-10-01"), y=1.05, label="New York, NY",size=18,fontface="bold") + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank()) + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())  + theme(legend.position = "none")
myPlots[[3]] <- myPlots[[3]] + ylim(0.5,1.05)  +  annotate('text',x=as.Date("2020-10-20"), y=1.05, label="Miami-Dade, FL",size=18,fontface="bold") +  theme(axis.title.x = element_blank()) + theme(legend.position = "none")
myPlots[[4]] <- myPlots[[4]] + ylim(0.5,1.05) +  annotate('text',x=as.Date("2020-09-01"), y=1.05, label="Dallas, TX",size=18,fontface="bold") + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank()) +  theme(axis.title.x = element_blank())  + theme(legend.position = "none")

pdf("Figures/CovidCoverage.pdf", width=30, height = 20)
grid.arrange(legend,blankplot,myPlots[[1]],myPlots[[2]],myPlots[[3]],myPlots[[4]],ncol=2,nrow=3,widths=c(3.5,3) , heights = c(0.4, 2.5,2.5),bottom=textGrob("Time", gp=gpar(fontsize=50)))
dev.off()

Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”
Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Warning message:
“Removed 5 rows containing missing values or values outside the scale range
(`geom_line()`).”
Warning message:
“Removed 7 rows containing missing values or values outside the scale range
(`geom_line()`).”


pdf 
  2

### Plotting Dynamic Eta

In [6]:
### Run FACI, AgACI, and MVP, and Plot Results
library(latex2exp)
library(cowplot)
library(grid)

alpha <- 0.1
gammaGrid <- c(0.001,0.002,0.004,0.008,0.0160,0.032,0.064,0.128)
Keps <- 2.12

myDates <- unique(res_final$time_value)[-1]
countiesToUse <- c("06075","36061","12086", "48113")  ## SF, NY, Miami-Dade, Dallas

allRes <- list()
allResDynamic <- list()
errMats <- list()
myPlots <- list()
count <- 1
for(geo in countiesToUse){
  allRes[[count]] <- conformalAdaptStable(allBetas[rownames(allBetas) ==geo],alpha,gammaGrid,sigma=1/1000,eta=2.72)
  allResDynamic[[count]] <- conformalAdaptStable(allBetas[rownames(allBetas) ==geo],alpha,gammaGrid,sigma=1/1000,eta=2.72,etaAdapt=TRUE)
  errMats[[count]] <- cbind(allRes[[count]][[6]],allResDynamic[[count]][[6]])
  myPlots[[count]] <- makeCoveragePlot(errMats[[count]],myDates,alpha,cols=c("Fixed","Dynamic"),colors=c(4,6),timeCutoff=200,smoothingWidth=200,lineTyping=TRUE,alphaC=1)
  myPlots[[count]] <- myPlots[[count]] + scale_colour_manual(name="Method",values = c("Fixed"=4,"Dynamic"=6),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Variable $\\eta$")),
                                                             aesthetics = c("color")) +
                    scale_linetype_manual(name="Method",values = c("Fixed" = "dashed", "Dynamic" = "solid"),
                                                             labels=c(TeX("DtACI Fixed $\\eta$"),TeX("DtACI Variable $\\eta$"))) + ylim(0.68,1)
  count <- count + 1
}
for(i in 1:length(countiesToUse)){
  myPlots[[i]] <- myPlots[[i]] + theme(axis.text=element_text(size=40),axis.title=element_text(size=50,face="plain"))
}

legend <- get_legend(myPlots[[1]]+theme(legend.position = "top",legend.text=element_text(size=60),legend.title=element_blank(),
                                     legend.key.width = unit(3,"line")))
blankplot <- ggplot()+geom_blank(aes(1,1)) + cowplot::theme_nothing()
myPlots[[1]] <- myPlots[[1]] + ylim(0.68,1.05) +  annotate('text',x=as.Date("2020-12-20"), y=1.05, label="San Francisco, CA",size=18,fontface="bold") + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank()) + theme(legend.position = "none")
myPlots[[2]] <- myPlots[[2]] + ylim(0.68,1.05)  +  annotate('text',x=as.Date("2020-11-18"), y=1.05, label="New York, NY",size=18,fontface="bold") + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank()) + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())  + theme(legend.position = "none")
myPlots[[3]] <- myPlots[[3]] + ylim(0.68,1.05)  +  annotate('text',x=as.Date("2020-11-30"), y=1.05, label="Miami-Dade, FL",size=18,fontface="bold") +  theme(axis.title.x = element_blank()) + theme(legend.position = "none")
myPlots[[4]] <- myPlots[[4]] + ylim(0.68,1.05) +  annotate('text',x=as.Date("2020-10-21"), y=1.05, label="Dallas, TX",size=18,fontface="bold") + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank()) +  theme(axis.title.x = element_blank())  + theme(legend.position = "none")

#options(repr.plot.width=25, repr.plot.height=15)
pdf("Figures/CovidCoverageVariableEta.pdf", width=30, height = 20)
grid.arrange(legend,blankplot,myPlots[[1]],myPlots[[2]],myPlots[[3]],myPlots[[4]],ncol=2,nrow=3,widths=c(3.5,3) , heights = c(0.4, 2.5,2.5),bottom=textGrob("Time", gp=gpar(fontsize=50)))
dev.off()

Warning message in get_plot_component(plot, "guide-box"):
“Multiple components found; returning the first one. To return all, use `return_all = TRUE`.”
Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Scale for y is already present.
Adding another scale for y, which will replace the existing scale.
Scale for y is already present.
Adding another scale for y, which will replace the existing scale.


pdf 
  2

### Plotting Case Counts

In [7]:
countiesToUse <- c("06075","36061","12086", "48113") 
datToPlot <- z[z$geo_value %in% c("06075","36061","12086", "48113") ,]
datToPlot$case <- datToPlot$case/1e-5 
myPlots <- list()
count <- 1
for(g in countiesToUse){
  myPlots[[count]] <- ggplot(datToPlot[datToPlot$geo_value ==g, ], aes(time_value)) + 
    geom_line(aes(y=case),size=4)  + theme_bw() +
    xlab("Date") + ylab("Seven Day Average Case Count") + theme(legend.position = "none") + ylim(0,0.008/1e-5) +
    theme(axis.text=element_text(size=30),axis.title=element_text(size=40,face="plain"))
  count <- count + 1
}
maxRangeY = 0.008/1e-5
myPlots[[1]] <- myPlots[[1]] +  annotate('text',x=as.Date("2020-09-21"), y=maxRangeY, label="San Francisco, CA",size=18,fontface="bold") + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank()) + theme(legend.position = "none")
myPlots[[2]] <- myPlots[[2]]  +  annotate('text',x=as.Date("2020-07-14"), y=maxRangeY, label="New York, NY",size=18,fontface="bold") + theme(axis.text.x = element_blank(),axis.ticks.x = element_blank(),axis.title.x = element_blank()) + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())  + theme(legend.position = "none")
myPlots[[3]] <- myPlots[[3]] +  annotate('text',x=as.Date("2020-08-27"), y=maxRangeY, label="Miami-Dade, FL",size=18,fontface="bold") + theme(legend.position = "none")
myPlots[[4]] <- myPlots[[4]] +  annotate('text',x=as.Date("2020-06-16"), y=maxRangeY, label="Dallas, TX",size=18,fontface="bold") + theme(axis.text.y = element_blank(),axis.ticks.y = element_blank(),axis.title.y = element_blank())  + theme(legend.position = "none")

pdf("Figures/CovidCaseCounts.pdf", width=30, height = 20)
grid.arrange(myPlots[[1]],myPlots[[2]],myPlots[[3]],myPlots[[4]],ncol=2,widths=c(3,3),heights=c(2,2.5))
dev.off()

pdf 
  2